In [1]:
import numpy as np
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor
# from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv("./weatherHistory.csv")


In [3]:
df.columns=df.columns.str.replace(" ","_")
df.head(2)

,Formatted_Date,Summary,Precip_Type,Temperature_(C),Apparent_Temperature_(C),Humidity,Wind_Speed_(km/h),Wind_Bearing_(degrees),Visibility_(km),Loud_Cover,Pressure_(millibars),Daily_Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.


In [4]:

df['Formatted_Date'] = pd.to_datetime(df['Formatted_Date'], utc=True)
df1 = df.copy()
df1['Year'] = df1['Formatted_Date'].dt.year
df1['Month'] = df1['Formatted_Date'].dt.month
df1['Day'] = df1['Formatted_Date'].dt.day
df1['Hour'] = df1['Formatted_Date'].dt.hour
df1['Minute'] = df1['Formatted_Date'].dt.minute
df1['Second'] = df1['Formatted_Date'].dt.second
df1.drop(columns=['Formatted_Date'], inplace=True)
column_order = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second'] + [col for col in df1.columns if col not in ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']]
df1 = df1[column_order]
print(df1)

       Year  Month  Day  Hour  Minute  Second        Summary Precip_Type  \
0      2006      3   31    22       0       0  Partly Cloudy        rain   
1      2006      3   31    23       0       0  Partly Cloudy        rain   
2      2006      4    1     0       0       0  Mostly Cloudy        rain   
3      2006      4    1     1       0       0  Partly Cloudy        rain   
4      2006      4    1     2       0       0  Mostly Cloudy        rain   
...     ...    ...  ...   ...     ...     ...            ...         ...   
96448  2016      9    9    17       0       0  Partly Cloudy        rain   
96449  2016      9    9    18       0       0  Partly Cloudy        rain   
96450  2016      9    9    19       0       0  Partly Cloudy        rain   
96451  2016      9    9    20       0       0  Partly Cloudy        rain   
96452  2016      9    9    21       0       0  Partly Cloudy        rain   

       Temperature_(C)  Apparent_Temperature_(C)  Humidity  Wind_Speed_(km/h)  \
0     

In [5]:
print(df1.head(2))

   Year  Month  Day  Hour  Minute  Second        Summary Precip_Type  \
0  2006      3   31    22       0       0  Partly Cloudy        rain   
1  2006      3   31    23       0       0  Partly Cloudy        rain   

   Temperature_(C)  Apparent_Temperature_(C)  Humidity  Wind_Speed_(km/h)  \
0         9.472222                  7.388889      0.89            14.1197   
1         9.355556                  7.227778      0.86            14.2646   

   Wind_Bearing_(degrees)  Visibility_(km)  Loud_Cover  Pressure_(millibars)  \
0                   251.0          15.8263         0.0               1015.13   
1                   259.0          15.8263         0.0               1015.63   

                       Daily_Summary  
0  Partly cloudy throughout the day.  
1  Partly cloudy throughout the day.  


In [6]:
df1= df1.drop('Daily_Summary',axis = 1)
df1 = df1.drop('Precip_Type',axis = 1)

In [7]:
arr = df1['Summary'].unique()
print(arr)

dict = {'Partly Cloudy':1, 'Mostly Cloudy':2, 'Overcast':3, 'Foggy':4,
 'Breezy and Mostly Cloudy':5, 'Clear':6,'Breezy and Partly Cloudy':7,
 'Breezy and Overcast':8, 'Humid and Mostly Cloudy':9,'Humid and Partly Cloudy':10,
 'Windy and Foggy':11, 'Windy and Overcast':12, 'Breezy and Foggy':13,
 'Windy and Partly Cloudy':14, 'Breezy':15, 'Dry and Partly Cloudy':16,
 'Windy and Mostly Cloudy':17, 'Dangerously Windy and Partly Cloudy':18, 'Dry':19,
 'Windy':20, 'Humid and Overcast':21,'Light Rain':22, 'Drizzle':23, 'Windy and Dry':24,
 'Dry and Mostly Cloudy':25, 'Breezy and Dry':26, 'Rain':27}

['Partly Cloudy' 'Mostly Cloudy' 'Overcast' 'Foggy'
 'Breezy and Mostly Cloudy' 'Clear' 'Breezy and Partly Cloudy'
 'Breezy and Overcast' 'Humid and Mostly Cloudy' 'Humid and Partly Cloudy'
 'Windy and Foggy' 'Windy and Overcast' 'Breezy and Foggy'
 'Windy and Partly Cloudy' 'Breezy' 'Dry and Partly Cloudy'
 'Windy and Mostly Cloudy' 'Dangerously Windy and Partly Cloudy' 'Dry'
 'Windy' 'Humid and Overcast' 'Light Rain' 'Drizzle' 'Windy and Dry'
 'Dry and Mostly Cloudy' 'Breezy and Dry' 'Rain']


In [8]:
df1['Summary'] = df1['Summary'].map(dict)

In [9]:
# Split the data into training and testing sets (80% for training, 20% for testing)

X = df1.drop('Temperature_(C)', axis=1)
y = df1['Temperature_(C)']

# print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a multiple linear regression model
model = LinearRegression()
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5)
adaBoost_model = AdaBoostRegressor(n_estimators=100)
extraTrees_model = ExtraTreesRegressor(n_estimators=100,random_state= 0)

model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
adaBoost_model.fit(X_train, y_train)
extraTrees_model.fit(X_train,y_train)


# Make predictions on the testing set
y_pred = model.predict(X_test)
rf_y_pred = rf_model.predict(X_test)
adaBoost_y_pred = adaBoost_model.predict(X_test)
extraTrees_y_pred = extraTrees_model.predict(X_test)
# Evaluate the model

rf_mse = mean_squared_error(y_test,rf_y_pred)
rf_mae = mean_absolute_error(y_test,rf_y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
adaBoost_mse = mean_squared_error(y_test,adaBoost_y_pred)
adaBoost_mae = mean_absolute_error(y_test,adaBoost_y_pred)
extraTrees_mse = mean_squared_error(y_test,extraTrees_y_pred)
extraTrees_mae = mean_absolute_error(y_test,extraTrees_y_pred)


print(f'MSE (Linear Regression): {mse:.2f}, MAE (Linear Regression): {mae:.2f}')
print(f'MSE (Random Forest): {rf_mse:.2f}, MAE (Random Forest): {rf_mae:.2f}')
print(f'MSE (Adaboost): {adaBoost_mse:.2f}, MAE (Adaboost): {adaBoost_mae:.2f}')
print(f'MSE (Extra Random Forests): {extraTrees_mse:.2f}, MAE (Extra Random Forests): {extraTrees_mae:.2f}')

MSE (Linear Regression): 0.90, MAE (Linear Regression): 0.74
MSE (Random Forest): 0.60, MAE (Random Forest): 0.51
MSE (Adaboost): 1.03, MAE (Adaboost): 0.82
MSE (Extra Random Forests): 0.00, MAE (Extra Random Forests): 0.02
